In [1]:
from utils import ReadTree
import pandas as pd
import numpy as np

In [2]:
# Parameters 
nodsegmmax = 15
outletPressure = 0.0

In [3]:
treeData, treeConnectivity = ReadTree('SmallNet_0.cco')
print(treeData[0])
Ids = [v[0] for v in treeData]

Reading SmallNet_0.cco
	Number of vessels in the tree: 415 

[0, 40.0, 60.0, 3750.0, -6000.0, 0.0, 0.0, -2250.0, 0.0, 0.0]


In [4]:
dflist = [[v[0], 2*v[1], v[3], v[2], v[4], v[5], v[6], v[7], v[8],v[9], v[0], v[0]+1] for v in treeData]
df = pd.DataFrame(dflist, columns=['Id', 'Diameter', 'Length', 'Flow', 'xprox','yprox','zprox',
                                   'xdist', 'ydist','zdist', 'From', 'To'])

scale = 1./10.
#df.Diameter *= scale
df.Length   *= scale
df.xprox    *= scale
df.yprox    *= scale
df.xdist    *= scale
df.ydist    *= scale
df.Flow     /= scale
df.head(-4)

,Id,Diameter,Length,Flow,xprox,yprox,zprox,xdist,ydist,zdist,From,To
0,0,80.000000,375.000000,600.000000,-600.000000,0.000000,0.0,-225.000000,0.000000,0.0,0,1
1,1,63.111727,39.000000,305.357143,-225.000000,0.000000,0.0,-225.000000,39.000000,0.0,1,2
2,2,58.332509,39.000000,230.357143,-225.000000,39.000000,0.0,-225.000000,78.000000,0.0,2,3
3,3,58.332509,39.000000,230.357143,-225.000000,78.000000,0.0,-225.000000,117.000000,0.0,3,4
4,4,58.332509,39.000000,230.357143,-225.000000,117.000000,0.0,-225.000000,156.000000,0.0,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
406,365,13.323200,17.947756,5.357143,-94.333333,-36.666667,0.0,-84.388211,-51.607098,0.0,365,366
407,366,20.770781,1.416394,16.071429,-94.333333,-36.666667,0.0,-92.944444,-36.388889,0.0,366,367
408,373,13.180135,16.551427,5.357143,-92.944444,-36.388889,0.0,-96.032687,-20.128123,0.0,373,374
409,374,18.516912,7.081972,10.714286,-92.944444,-36.388889,0.0,-86.000000,-35.000000,0.0,374,375


In [5]:
nodesProx = {v[0]:v[4:7] for v in treeData}
nodesDist = {v[0]+1:v[7:] for v in treeData}

nodesDist[0] = nodesProx[0]
outletNodes = []

for v in treeConnectivity[1:]:
    isOutlet = False
    if len(v[-1])==0:
        isOutlet = True
    IdDistNode, IdProxNode = v[0]+1, v[1]+1
    df.at[v[0], 'From'] = IdProxNode
    df.at[v[0], 'To']   = IdDistNode
    
    if isOutlet and (IdDistNode not in outletNodes):
        outletNodes.append(IdDistNode)
print(f'There are {len(outletNodes)} outlet nodes.')

There are 112 outlet nodes.


In [6]:
f = open('Network.dat', 'w')
f.write('Network for patient1, sim0, stage0\n')
#f.write('8500 4000 20\t box dimensions in microns\n')
f.write(f'{df.xdist.max()-df.xprox.min()} {df.ydist.max()-df.yprox.min()} 20\n')
f.write('150 150 1\t number of tissue points in x,y,z directions\n')
f.write(f'{df.Length.max()}\t outer bound distance\n')
f.write(f'{df.Length.max()/nodsegmmax}\t max. segment length.  (5. in test case)\n')
f.write(f'{nodsegmmax}  \t nodsegm, max. allowed number of segments per node\n')
f.write(f'{1+int(df.Id.max())}\t total number of segments\n')
f.write('name\t type\t from\t to\t diam\t\t flow\t\t HD\n')

39

In [7]:
for index, row in df.iterrows():
    f.write(f'{index}\t 5\t {int(row.From)}\t {int(row.To)}\t {row.Diameter}\t\t {row.Flow}\t 0.4 *\n')

In [8]:
f.write(f'{len(nodesDist)} total number of nodes\n')
f.write('name\t x\t y\t z\n')
for i in range(len(nodesDist)):
    x,y,z = nodesDist[i]
    f.write(f'{i}\t {x}\t {y}\t 10*\n')

f.write(f'{1+len(outletNodes)} total number of boundary nodes\n')
f.write('node\t bctyp\t press/flow\t HD\t PO2\t solute\n')
#f.write(f'0\t 2\t 40.09068\t 0.4\t 40.0\t *\n')
f.write(f'0\t 2\t {df.Flow.max()}\t 0.4\t 40.0\t *\n')
#f.write(f'0\t 0\t 50\t 0.4\t 40.0\t *\n')

for outletNode in outletNodes:
    f.write(f'{outletNode}\t 0\t {outletPressure}\t 0.4\t 40.0\t *\n')

In [9]:
f.close()